In [3]:
import functools
import nest_asyncio
import os
import traceback
import pandas as pd
import telebot
from icecream import ic
from IPython import get_ipython
from IPython.utils.capture import capture_output
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain.chat_models import ChatOpenAI
from typing import TypedDict, Annotated, Sequence
import operator
from langchain import hub
import importlib
import json

# Apply nest_asyncio
nest_asyncio.apply()

# Load configuration from config.json
with open('config.json') as config_file:
    config = json.load(config_file)

os.environ["LANGCHAIN_API_KEY"] = config["LANGCHAIN_API_KEY"]
os.environ["OPENAI_API_KEY"] = config["OPENAI_API_KEY"]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "nlmk_bot"
TELEGRAMM_API_KEY = config["TELEGRAM_BOT_TOKEN"]

# Import and reload the gsheet_handler module
module = importlib.import_module('gsheet_handler')
importlib.reload(module)
ggl = module.GSheetHandler('google_sheet_key.json')

# Extract data from Google Sheets
example_dialog = ggl.extract_data_from_google_sheet('https://docs.google.com/spreadsheets/d/1sBAFF62fAjSdPrGYTWgpjmBjLRiFCb8DNAX8ZypI-_U/edit?gid=0#gid=0', 0)
example_dialog.to_csv('example_dialog.csv', index=False)

support = ggl.extract_data_from_google_sheet('https://docs.google.com/spreadsheets/d/1C3m50L3iUWhkkrp5CWEmksVRAkA8y4hX6m2tYgS8-4I/edit?gid=1891107571#gid=1891107571', 0)
support.to_csv('support.csv', index=False)

# New Google Sheets link for "помещения и сооружения"
facilities = ggl.extract_data_from_google_sheet('https://docs.google.com/spreadsheets/d/19oNWcJstB1fJRqJAbi1YKygUlMqs91fF8-Jw-OTvPXo/edit?gid=362855495#gid=362855495', 0)
facilities.to_csv('facilities.csv', index=False)

# New Google Sheets link for "особые экономические зоны и технопарки"
special_economic_zones = ggl.extract_data_from_google_sheet('https://docs.google.com/spreadsheets/d/1p24i_RrKPCScLhfMNcw5C61t4iWeOpIziG45rukY9TY/edit?usp=sharing', 0)
special_economic_zones.to_csv('special_economic_zones.csv', index=False)

from langchain_experimental.agents import create_csv_agent
from langchain.llms import OpenAI
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.agents import AgentType

# Function to run agent with a query and list of CSV files
def run_agent(query, csv_files):
    # Create a CSV agent using LangChain and OpenAI with the provided CSV files
    agent = create_csv_agent(
        ChatOpenAI(temperature=0, model="gpt-4o"),
        csv_files,
        verbose=True,
        agent_type=AgentType.OPENAI_FUNCTIONS,
        allow_dangerous_code=True,  # Opt-in to use the functionality
    )
    return agent.invoke(query)

# Example usage
csv_files = ["example_dialog.csv", "support.csv", "facilities.csv", "special_economic_zones.csv"]
print(run_agent("Сколько у тебя таблиц?", csv_files))

no docx


> Entering new AgentExecutor chain...
У меня есть 5 таблиц.

> Finished chain.
{'input': 'Сколько у тебя таблиц?', 'output': 'У меня есть 5 таблиц.'}


In [4]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from datetime import datetime

# Функция, возвращающая сегодняшнюю дату, принимает один аргумент
def get_today_date(input):
    return str(datetime.today().date())

# Создаем инструмент на базе нашей функции
date_tool = Tool(
    name="GetTodayDate",
    func=get_today_date,
    description="Returns today's date in YYYY-MM-DD format."
)

# Список инструментов
tools = [date_tool]


from langchain import hub
prompt = hub.pull("mlenparrot/invest_mos_bot")

llm = ChatOpenAI(model="gpt-3.5-turbo")
agent = initialize_agent(tools, llm, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, prompt=prompt)

# Взаимодействие с агентом
response = agent.run("Ты кто?")
print(response)




> Entering new AgentExecutor chain...
I need to provide my identity.
Action: 
Action Input: GetTodayDate(input)
Observation:  is not a valid tool, try one of [GetTodayDate].
Thought:I need to use the correct tool to provide my identity.
Action: GetTodayDate
Action Input: 
Observation: 2024-06-16
Thought:I can provide my identity now with today's date.
Final Answer: I am from the future, it is 2024-06-16.

> Finished chain.
I am from the future, it is 2024-06-16.
